In [54]:
import time
import datetime
import urllib.request, json
import requests

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

from oceanoobsbrasil.db import GetData
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
url = 'http://osmc.noaa.gov/erddap/tabledap/OSMC_30day.htmlTable?platform_code%2Cplatform_type%2Ccountry%2Ctime%2Clatitude%2Clongitude%2Cobservation_depth%2Csst%2Catmp%2Cprecip%2Cztmp%2Czsal%2Cslp%2Cwindspd%2Cwinddir%2Cwvht%2Cwaterlevel%2Cclouds%2Cdewpoint%2Cuo%2Cvo%2Cwo%2Crainfall_rate%2Chur%2Csea_water_elec_conductivity%2Csea_water_pressure%2Crlds%2Crsds%2Cwaterlevel_met_res%2Cwaterlevel_wrt_lcd%2Cwater_col_ht%2Cwind_to_direction%2Clon360&platform_type=%22DRIFTING%20BUOYS%20(GENERIC)%22&time%3E=2022-07-21T00%3A00%3A00Z&latitude%3E=-89&latitude%3C=15&longitude%3E=-90&longitude%3C=20'
url = 'http://osmc.noaa.gov/erddap/tabledap/OSMC_30day.htmlTable?platform_code%2Cplatform_type%2Ccountry%2Ctime%2Clatitude%2Clongitude%2Cobservation_depth%2Csst%2Catmp%2Cprecip%2Cztmp%2Czsal%2Cslp%2Cwindspd%2Cwinddir%2Cwvht%2Cwaterlevel%2Cclouds%2Cdewpoint%2Cuo%2Cvo%2Cwo%2Crainfall_rate%2Chur%2Csea_water_elec_conductivity%2Csea_water_pressure%2Crlds%2Crsds%2Cwaterlevel_met_res%2Cwaterlevel_wrt_lcd%2Cwater_col_ht%2Cwind_to_direction%2Clon360&platform_type=%22PROFILING%20FLOATS%20AND%20GLIDERS%20(GENERIC)%22&time%3E=2022-07-21T00%3A00%3A00Z&latitude%3E=-89&latitude%3C=15&longitude%3E=-90&longitude%3C=20&observation_depth%3C=1'
resp = requests.get(url)


In [63]:
df = pd.read_html(resp.content)
df = df[1]
df.columns = df.columns.droplevel(level=1)

In [67]:
df = df.loc[df['observation_depth']>=-1]
df.sst.fillna(df.ztmp.dropna(), inplace=True)
df = df[['time', 'latitude', 'longitude','sst', 'atmp', 'ztmp', 'wvht']]


In [70]:
df.head()

,time,latitude,longitude,sst,atmp,ztmp,slp,wvht
0,2022-07-21T00:00:00Z,-38.014023,12.428475,NaN,NaN,16.399012,NaN,NaN
1,2022-07-21T00:00:00Z,-38.014023,12.428475,NaN,NaN,16.402002,NaN,NaN
2,2022-07-21T00:00:00Z,11.808838,-44.432163,NaN,NaN,27.135004,NaN,NaN
3,2022-07-21T00:00:00Z,11.808838,-44.432163,NaN,NaN,27.136011,NaN,NaN
4,2022-07-21T04:36:00Z,-47.755040,-3.008395,NaN,NaN,5.328027,NaN,NaN


In [71]:
df.sst.fillna(df.ztmp.dropna(), inplace=True)

In [24]:
df.columns = ['date_time', 'lat', 'lon', 'sst', 'atmp', 'ztmp' 'pres', 'swvht']

In [41]:
datetime.strftime(datetime.now()-timedelta(days=1), format='%Y-%m-%dT00:00:00Z')

'2022-07-21T00:00:00Z'

In [30]:
df['date_time'] = pd.to_datetime(df['date_time'], format="%Y-%m-%dT%H:%M:%SZ")

In [46]:
df.fillna(np.nan)

,date_time,lat,lon,sst,atmp,pres,swvht
0,2022-07-21 00:00:00,-39.6268,7.8202,12.59,NaN,1017.9,NaN
1,2022-07-21 00:00:00,3.0739,8.5073,27.05,NaN,1012.5,NaN
2,2022-07-21 00:00:00,5.7872,0.3541,NaN,NaN,NaN,NaN
3,2022-07-21 00:00:00,0.8735,-46.5852,28.10,NaN,1012.1,NaN
4,2022-07-21 00:00:00,-34.4136,-36.2684,16.92,NaN,1028.4,NaN
...,...,...,...,...,...,...,...
8323,2022-07-22 18:00:00,5.1214,3.9824,27.12,NaN,NaN,NaN
8324,2022-07-22 18:00:00,8.7146,-13.7270,27.49,NaN,NaN,NaN
8325,2022-07-22 18:00:00,-24.3070,4.1776,18.68,NaN,NaN,NaN
8326,2022-07-22 18:00:00,-37.7576,-0.0514,13.55,NaN,1029.2,NaN


In [72]:
from oceanoobsbrasil.others.drifters import Drifter

In [80]:
d = Drifter(platform_type='PROFILING FLOATS AND GLIDERS (GENERIC)')

In [81]:
d.get()

In [61]:
d.result.describe()

,lat,lon,sst,atmp,pres,swvht
count,27434.000000,27434.000000,23687.000000,148.000000,20499.000000,15.000000
mean,-23.782761,-27.792121,18.288439,-7.460811,1015.765837,1.684667
std,20.821604,26.298900,7.918611,6.684571,11.444044,0.228313
min,-63.278600,-89.998400,-4.950000,-20.800000,954.100000,1.420000
25%,-38.256000,-41.342450,13.850000,-14.725000,1012.300000,1.580000
50%,-29.415400,-24.811600,18.720000,-4.100000,1017.000000,1.630000
75%,-4.953000,-9.302200,26.110000,-1.600000,1023.600000,1.680000
max,14.999730,17.768380,35.930000,0.800000,1034.600000,2.390000


In [10]:
db = Getdata()
df = db.get(table='data_no_stations', institution=['in', ['ndbc']], date_time=['>', '2021-09-12'])
df.shape

(139, 17)

In [11]:
type(df.shape)

tuple